In [ ]:
import cv2
import numpy as np
from scipy.spatial.transform import Rotation as R


def generate_pixel_to_world_map(K, D, R_wc, t_wc, image_size, plane_z=0.0):
    H, W = image_size
    xyz_map = np.zeros((H, W, 3), dtype=np.float32)

    # 像素坐标网格
    u, v = np.meshgrid(np.arange(W), np.arange(H))  # shape: (H, W)
    pixel_coords = np.stack([u, v], axis=-1).astype(np.float32)  # (H, W, 2)
    pixel_coords_flat = pixel_coords.reshape(-1, 1, 2)

    # 去畸变得到归一化相机坐标
    undistorted_pts = cv2.undistortPoints(pixel_coords_flat, K, D)  # (N,1,2)
    undistorted_pts = undistorted_pts.reshape(-1, 2)

    # 单位射线
    rays_cam = np.concatenate(
        [undistorted_pts, np.ones((undistorted_pts.shape[0], 1))], axis=1
    )

    # 相机坐标系 → 世界坐标系的射线方向
    rays_world = (R_wc @ rays_cam.T).T  # (N,3)

    # ✅ 正确计算相机中心
    cam_center = t_wc.flatten()  # shape (3,)

    # 与 z=plane_z 的平面求交
    t_vals = (plane_z - cam_center[2]) / rays_world[:, 2]
    pts_world = cam_center + rays_world * t_vals[:, np.newaxis]

    xyz_map = pts_world.reshape(H, W, 3).astype(np.float32)
    return xyz_map


def save_xyz_map_binary(xyz_map: np.ndarray, file_path: str):
    # 确保是 float32 类型，等价于 CV_32FC3
    if xyz_map.dtype != np.float32:
        xyz_map = xyz_map.astype(np.float32)

    # 写入二进制文件
    with open(file_path, "wb") as f:
        f.write(xyz_map.tobytes())


def load_xyz_map_binary(
    file_path: str, height: int = 480, width: int = 640
) -> np.ndarray:
    """
    加载用 save_xyz_map_binary 保存的二进制XYZ地图

    参数:
        file_path: 二进制文件路径
        height: 原始数组的高度（行数）
        width: 原始数组的宽度（列数）

    返回:
        xyz_map: 形状为 (height, width, 3) 的 np.float32 数组
    """
    # 读取二进制数据
    with open(file_path, "rb") as f:
        data = f.read()

    # 转换为np.float32数组，并reshape为 (height, width, 3)
    xyz_map = np.frombuffer(data, dtype=np.float32).reshape(height, width, 3)

    return xyz_map


def compute_world_projection_errors(img_points, obj_points, xyz_map):
    """
    计算像素反投影到世界坐标后，与原始3D目标点之间的欧式距离误差。

    参数:
    - img_points: (N, 2) 图像中的像素点，二维数组。
    - obj_points: (N, 3) 对应的世界坐标点，三维数组。
    - xyz_map: (H, W, 3) 每个像素对应的世界坐标反投影结果。

    返回:
    - dist_errors: 每个点的欧式距离误差（单位：米）
    - mean_error: 平均误差
    """
    reprojected_world_points = []

    for px in img_points:
        u, v = int(round(px[0])), int(round(px[1]))
        if 0 <= v < xyz_map.shape[0] and 0 <= u < xyz_map.shape[1]:
            pt_world = xyz_map[v, u]  # 注意顺序是[y, x]
            reprojected_world_points.append(pt_world)
        else:
            reprojected_world_points.append([np.nan, np.nan, np.nan])  # 越界处理

    reprojected_world_points = np.array(reprojected_world_points)
    dist_errors = np.linalg.norm(obj_points - reprojected_world_points, axis=1)
    mean_error = np.nanmean(dist_errors)

    return dist_errors, mean_error

In [ ]:
# front
# 假设你已有内参 K 和畸变参数 D（5 个）
K = np.array([[571.0, 0.0, 329.86688232], [0.0, 571.0, 239.08282471], [0.0, 0.0, 1.0]])
D = np.array([0.0, 0.0, 0.0, 0.0, 0.0])

# 世界坐标系中的棋盘格角点（Z=0）
obj_points = np.array(
    [
        [2.50, 0.288, -0.07],
        [2.50, -0.612, -0.07],
        [1.60, -0.612, -0.07],
        [1.60, 0.288, -0.07],
    ],
    dtype=np.float32,
)

# 图像中的像素坐标（c）
img_points = np.array(
    [
        [270.0, 290.0],
        [507.0, 295.0],
        [607.0, 465.0],
        [224.0, 456.0],
    ],
    dtype=np.float32,
)


# # 2. 读取图像
# image = cv2.imread('/home/ubuntu/Desktop/project/utils_python/tmp/2025-07-28-14-35-20/nuwa_1_rgb0_image/1753684520_973729249.jpg')  # 替换为你的图像路径
# # 1. 加载 ArUco 字典和检测器
# aruco_dict = cv2.aruco.getPredefinedDictionary(cv2.aruco.DICT_4X4_50)
# aruco_parameters = cv2.aruco.DetectorParameters()
# detector = cv2.aruco.ArucoDetector(aruco_dict, aruco_parameters)
# gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
# corners, ids, _ = detector.detectMarkers(gray)
# print(corners)
# # 4. 检查是否检测到 ArUco 标记
# if len(corners) > 0:
#     print("检测到 ArUco 标记！")
#     # 选择你感兴趣的标记角点（如果有多个标记）
#     img_points = corners[0][0]  # 获取第一个标记的角点（例如 4 个角点）

#     # 可视化检测到的 ArUco 标记
#     cv2.aruco.drawDetectedMarkers(image, corners, ids)
#     cv2.imwrite('ArUco_Detect.jpg', image)
# else:
#     print("没有检测到 ArUco 标记")


# 第一步：将畸变像素点转换为去畸变的归一化点
# 返回去畸变后的像素点
undistorted_pts = cv2.undistortPoints(img_points.reshape(-1, 1, 2), K, D, P=K)
# # reshape成 Nx2
undistorted_pts = undistorted_pts.reshape(-1, 2)
print("去畸变后的像素点：")
print(undistorted_pts)
# undistorted_pts = img_points.reshape(-1, 1, 2).reshape(-1, 2)
# flags=cv2.SOLVEPNP_ITERATIVE
# 第二步：调用普通solvePnP，传入去畸变后的像素点和无畸变的内参
retval, rvec, tvec = cv2.solvePnP(
    obj_points, undistorted_pts, K, distCoeffs=None, flags=cv2.SOLVEPNP_ITERATIVE
)
R_mat, _ = cv2.Rodrigues(rvec)
rotation = R.from_matrix(R_mat)
euler_angles = rotation.as_euler("ZYX", degrees=True)  # 输出 roll, pitch, yaw，单位是度
print(euler_angles)
# ✅ 验证：将 obj_points 投影回图像
# 注意：此时不带畸变，所以 distCoeffs=None
img_points_proj, _ = cv2.projectPoints(obj_points, rvec, tvec, K, distCoeffs=None)
img_points_proj = img_points_proj.reshape(-1, 2)
print("投影点：")
print(img_points_proj)
# 计算重投影误差
errors = np.linalg.norm(undistorted_pts - img_points_proj, axis=1)
mean_error = np.mean(errors)

print("\n重投影误差 (单位：像素):")
for i, err in enumerate(errors):
    print(f"  点{i + 1}: 误差 = {err:.3f} px")
print(f"  平均误差 = {mean_error:.3f} px")

R_cam, _ = cv2.Rodrigues(rvec)
print(R_cam)
print(tvec)
R_wc = R_cam.T
t_wc = -R_cam.T @ tvec
print(R_wc)
print(t_wc)
# t_wc[0] = t_wc[0] +0.10


image_size = (480, 640)  # H, W
xyz_map = generate_pixel_to_world_map(K, D, R_wc, t_wc, image_size)

# 查看某像素的世界坐标
print("像素坐标对应的世界坐标:")
for px in img_points:
    u, v = int(round(px[0])), int(round(px[1]))
    print("Pixel (", v, ",", u, ") →", xyz_map[v, u])  # (X, Y, Z)
# save_xyz_map_binary(xyz_map, 'front_map.bin')

# 调用函数
dist_errors, mean_error = compute_world_projection_errors(
    img_points, obj_points, xyz_map
)

# 打印结果
print("\n📏 3D 世界坐标误差 (单位：米):")
for i, err in enumerate(dist_errors):
    print(f"  点 {i+1}: 距离误差 = {err:.4f} m")
print(f"  平均距离误差 = {mean_error:.4f} m")

# img_points = np.array(
#     [[1605, 695], [1454, 545], [1359, 468], [1250, 395]], dtype=np.float32
# )

# # 世界坐标系中的棋盘格角点（Z=0）
# obj_points = np.array(
#     [
#         [1.06, -0.15 - 1.7485, 0],
#         [1.90, -0.15 - 1.7485, 0],
#         [2.61, -0.15 - 1.7485, 0],
#         [3.85, -0.15 - 1.7485, 0],
#     ],
#     dtype=np.float32,
# )
# 调用函数
dist_errors, mean_error = compute_world_projection_errors(
    img_points, obj_points, xyz_map
)

# 打印结果
print("\n📏 3D 世界坐标误差 (单位：米):")
for i, err in enumerate(dist_errors):
    print(f"  点 {i+1}: 距离误差 = {err:.4f} m")
print(f"  平均距离误差 = {mean_error:.4f} m")

去畸变后的像素点：
[[270. 290.]
 [507. 295.]
 [607. 465.]
 [224. 456.]]
[-85.68520063 -69.3175762  176.37825025]
投影点：
[[270.68274 290.295  ]
 [506.20786 294.58282]
 [607.26984 465.27927]
 [223.79805 455.79422]]

重投影误差 (单位：像素):
  点1: 误差 = 0.744 px
  点2: 误差 = 0.895 px
  点3: 误差 = 0.388 px
  点4: 误差 = 0.288 px
  平均误差 = 0.579 px
[[ 0.02657255 -0.99962055  0.00725659]
 [-0.35218684 -0.01615528 -0.93579027]
 [ 0.93555242  0.02231066 -0.35248249]]
[[-0.00566246]
 [ 1.01658313]
 [-0.17384156]]
[[ 0.02657255 -0.35218684  0.93555242]
 [-0.99962055 -0.01615528  0.02231066]
 [ 0.00725659 -0.93579027 -0.35248249]]
[[0.52081555]
 [0.01464139]
 [0.89007359]]
[[ 0.02657255 -0.35218684  0.93555242]
 [-0.99962055 -0.01615528  0.02231066]
 [ 0.00725659 -0.93579027 -0.35248249]]
[[0.52081555]
 [0.01464139]
 [0.89007359]]
像素坐标对应的世界坐标:
Pixel ( 290 , 270 ) → [2.3579967  0.27079785 0.        ]
Pixel ( 295 , 507 ) → [ 2.3524542  -0.56824243  0.        ]
Pixel ( 465 , 607 ) → [ 1.5221452  -0.56608343  0.        ]
Pixel ( 

In [95]:
# front
# 假设你已有内参 K 和畸变参数 D（5 个）
K = np.array([[571.0, 0.0, 329.86688232], [0.0, 571.0, 239.08282471], [0.0, 0.0, 1.0]])
D = np.array([0.0, 0.0, 0.0, 0.0, 0.0])

# 世界坐标系中的棋盘格角点（Z=0）
obj_points = np.array(
    [
        [2.50, 0.31, -0.07],
        [2.50, -0.59, -0.07],
        [1.60, -0.59, -0.07],
        [1.60, 0.31, -0.07],
    ],
    dtype=np.float32,
)

# 图像中的像素坐标（c）
img_points = np.array(
    [
        [262, 291],
        [499, 296.0],
        [595, 464.0],
        [212, 457.0],
    ],
    dtype=np.float32,
)


# # 2. 读取图像
# image = cv2.imread('/home/ubuntu/Desktop/project/utils_python/tmp/2025-07-28-17-01-49/nuwa_1_rgb0_image/1753693310_795626747.jpg')  # 替换为你的图像路径
# # 1. 加载 ArUco 字典和检测器
# aruco_dict = cv2.aruco.getPredefinedDictionary(cv2.aruco.DICT_4X4_50)
# aruco_parameters = cv2.aruco.DetectorParameters()
# detector = cv2.aruco.ArucoDetector(aruco_dict, aruco_parameters)
# gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
# corners, ids, _ = detector.detectMarkers(gray)
# print(corners)
# # 4. 检查是否检测到 ArUco 标记
# if len(corners) > 0:
#     print("检测到 ArUco 标记！")
#     # 选择你感兴趣的标记角点（如果有多个标记）
#     img_points = corners[0][0]  # 获取第一个标记的角点（例如 4 个角点）

#     # 可视化检测到的 ArUco 标记
#     cv2.aruco.drawDetectedMarkers(image, corners, ids)
#     cv2.imwrite('ArUco_Detect.jpg', image)
# else:
#     print("没有检测到 ArUco 标记")


# 第一步：将畸变像素点转换为去畸变的归一化点
# 返回去畸变后的像素点
undistorted_pts = cv2.undistortPoints(img_points.reshape(-1, 1, 2), K, D, P=K)
# # reshape成 Nx2
undistorted_pts = undistorted_pts.reshape(-1, 2)
print("去畸变后的像素点：")
print(undistorted_pts)
# undistorted_pts = img_points.reshape(-1, 1, 2).reshape(-1, 2)
# flags=cv2.SOLVEPNP_ITERATIVE
# 第二步：调用普通solvePnP，传入去畸变后的像素点和无畸变的内参
retval, rvec, tvec = cv2.solvePnP(
    obj_points, undistorted_pts, K, distCoeffs=None, flags=cv2.SOLVEPNP_ITERATIVE
)
R_mat, _ = cv2.Rodrigues(rvec)
rotation = R.from_matrix(R_mat)
euler_angles = rotation.as_euler("ZYX", degrees=True)  # 输出 roll, pitch, yaw，单位是度
print(euler_angles)
# ✅ 验证：将 obj_points 投影回图像
# 注意：此时不带畸变，所以 distCoeffs=None
img_points_proj, _ = cv2.projectPoints(obj_points, rvec, tvec, K, distCoeffs=None)
img_points_proj = img_points_proj.reshape(-1, 2)
print("投影点：")
print(img_points_proj)
# 计算重投影误差
errors = np.linalg.norm(undistorted_pts - img_points_proj, axis=1)
mean_error = np.mean(errors)

print("\n重投影误差 (单位：像素):")
for i, err in enumerate(errors):
    print(f"  点{i + 1}: 误差 = {err:.3f} px")
print(f"  平均误差 = {mean_error:.3f} px")

R_cam, _ = cv2.Rodrigues(rvec)
print(R_cam)
print(tvec)
R_wc = R_cam.T
t_wc = -R_cam.T @ tvec
print(R_wc)
print(t_wc)


image_size = (480, 640)  # H, W

xyz_map = generate_pixel_to_world_map(K, D, R_wc, t_wc, image_size, plane_z=-0.07)

# 查看某像素的世界坐标
print("像素坐标对应的世界坐标:")
for px in img_points:
    u, v = int(round(px[0])), int(round(px[1]))
    print("Pixel (", v, ",", u, ") →", xyz_map[v, u])  # (X, Y, Z)
# save_xyz_map_binary(xyz_map, 'front_map.bin')

# 调用函数
dist_errors, mean_error = compute_world_projection_errors(
    img_points, obj_points, xyz_map
)

# 打印结果
print("\n📏 3D 世界坐标误差 (单位：米):")
for i, err in enumerate(dist_errors):
    print(f"  点 {i+1}: 距离误差 = {err:.4f} m")
print(f"  平均距离误差 = {mean_error:.4f} m")

# img_points = np.array(
#     [[1605, 695], [1454, 545], [1359, 468], [1250, 395]], dtype=np.float32
# )

# # 世界坐标系中的棋盘格角点（Z=0）
# obj_points = np.array(
#     [
#         [1.06, -0.15 - 1.7485, 0],
#         [1.90, -0.15 - 1.7485, 0],
#         [2.61, -0.15 - 1.7485, 0],
#         [3.85, -0.15 - 1.7485, 0],
#     ],
#     dtype=np.float32,
# )
# 调用函数
dist_errors, mean_error = compute_world_projection_errors(
    img_points, obj_points, xyz_map
)

R_wc = np.array(
    [
        [-0.013148, -0.349717, 0.936763],
        [-0.999875, 0.012850, -0.009237],
        [-0.008807, -0.936767, -0.349842],
    ]
)
t_wc = np.array([[0.440591], [0.029158], [0.978436]])
# print(R_wc)
# print(t_wc)
# R_cw = R_wc.T
# t_cw = -np.dot(R_cw, t_wc)
# print(R_cw)
# print(t_cw)
# t_cw[0]=t_cw[0]+0.06
# print(t_cw)

# t_wc = -np.dot(R_wc, t_cw)


image_size = (480, 640)  # H, W

xyz_map = generate_pixel_to_world_map(K, D, R_wc, t_wc, image_size, plane_z=-0.07)
save_xyz_map_binary(xyz_map, "front_map_test.bin")
dist_errors, mean_error = compute_world_projection_errors(
    img_points, obj_points, xyz_map
)
# 打印结果
print("\n📏 3D 世界坐标误差 (单位：米):")
for i, err in enumerate(dist_errors):
    print(f"  点 {i+1}: 距离误差 = {err:.4f} m")
print(f"  平均距离误差 = {mean_error:.4f} m")

去畸变后的像素点：
[[262. 291.]
 [499. 296.]
 [595. 464.]
 [212. 457.]]
[-86.11960482 -69.61753916 176.66047256]
投影点：
[[262.69196 291.66818]
 [498.05017 295.19556]
 [595.3342  464.54773]
 [211.82489 456.53284]]

重投影误差 (单位：像素):
  点1: 误差 = 0.962 px
  点2: 误差 = 1.245 px
  点3: 误差 = 0.642 px
  点4: 误差 = 0.499 px
  平均误差 = 0.837 px
[[ 0.02356982 -0.99970862  0.00520982]
 [-0.34748667 -0.0130787  -0.93759371]
 [ 0.93738865  0.02028857 -0.34769368]]
[[-0.00698444]
 [ 1.00936674]
 [-0.17820746]]
[[ 0.02356982 -0.34748667  0.93738865]
 [-0.99970862 -0.0130787   0.02028857]
 [ 0.00520982 -0.93759371 -0.34769368]]
[[0.51795576]
 [0.00983438]
 [0.88445068]]
像素坐标对应的世界坐标:
Pixel ( 291 , 262 ) → [ 2.505819    0.31345594 -0.07      ]
Pixel ( 296 , 499 ) → [ 2.4931056  -0.59185994 -0.07      ]
Pixel ( 464 , 595 ) → [ 1.601784 -0.589954 -0.07    ]
Pixel ( 457 , 212 ) → [ 1.5984535   0.30924624 -0.07      ]

📏 3D 世界坐标误差 (单位：米):
  点 1: 距离误差 = 0.0068 m
  点 2: 距离误差 = 0.0071 m
  点 3: 距离误差 = 0.0018 m
  点 4: 距离误差 = 0.0017 m

In [ ]:
# front
# 假设你已有内参 K 和畸变参数 D（5 个）
K = np.array([[571.0, 0.0, 329.86688232], [0.0, 571.0, 239.08282471], [0.0, 0.0, 1.0]])
D = np.array([0.0, 0.0, 0.0, 0.0, 0.0])
image_size = (480, 640)  # H, W
roll, pitch, yaw = 69.36901996548279, 179.9543793453007, 91.40716490747336
r = R.from_euler("ZYX", [yaw, pitch, roll], degrees=True)

# 获取对应的旋转矩阵
R_wc = r.as_matrix()
print(R_wc)
# R_wc = np.array(
#     [
#         [-0.013148, -0.349717, 0.936763],
#         [-0.999875, 0.012850, -0.009237],
#         [-0.008807, -0.936767, -0.349842],
#     ]
# )
t_wc = np.array([[0.4752837 + 0.06], [-0.0002232025], [0.9815271]])
xyz_map = generate_pixel_to_world_map(K, D, R_wc, t_wc, image_size, plane_z=0.0)

save_xyz_map_binary(xyz_map, "front_map_808.bin")
# 世界坐标系中的棋盘格角点（Z=0）
obj_points = np.array(
    [
        [2.50, 0.31, -0.0],
        [2.50, -0.59, -0.0],
        [1.60, -0.59, -0.0],
        [1.60, 0.31, -0.0],
    ],
    dtype=np.float32,
)

# 图像中的像素坐标（c）
img_points = np.array(
    [
        [262, 291],
        [499, 296.0],
        [595, 464.0],
        [212, 457.0],
    ],
    dtype=np.float32,
)
dist_errors, mean_error = compute_world_projection_errors(
    img_points, obj_points, xyz_map
)
# 打印结果
print("\n📏 3D 世界坐标误差 (单位：米):")
for i, err in enumerate(dist_errors):
    print(f"  点 {i+1}: 距离误差 = {err:.4f} m")
print(f"  平均距离误差 = {mean_error:.4f} m")

[[ 2.45571841e-02 -3.52259768e-01  9.35580034e-01]
 [-9.99698110e-01 -7.90772786e-03  2.32627839e-02]
 [-7.96230547e-04 -9.35868860e-01 -3.52347616e-01]]

📏 3D 世界坐标误差 (单位：米):
  点 1: 距离误差 = 0.0570 m
  点 2: 距离误差 = 0.0358 m
  点 3: 距离误差 = 0.0379 m
  点 4: 距离误差 = 0.0370 m
  平均距离误差 = 0.0419 m


In [ ]:
# back
# 假设你已有内参 K 和畸变参数 D（5 个）
K = np.array(
    [
        [566.72375488, 0.0, 333.63687134],
        [0.0, 566.10162354, 230.97738647],
        [0.0, 0.0, 1.0],
    ]
)
D = np.array([0.0, 0.0, 0.0, 0.0, 0.0])
image_size = (480, 640)  # H, W
roll, pitch, yaw = -116.1267943062868, 1.878901157409764, 90.93493245844785
r = R.from_euler("ZYX", [yaw, pitch, roll], degrees=True)

# 获取对应的旋转矩阵
R_wc = r.as_matrix()
t_wc = np.array([[-0.1393937 + 0.06], [0.009496391], [0.878864]])
xyz_map = generate_pixel_to_world_map(K, D, R_wc, t_wc, image_size, plane_z=0.0)

save_xyz_map_binary(xyz_map, "back_map_808.bin")
# 世界坐标系中的棋盘格角点（Z=0）

In [ ]:
# left
# 假设你已有内参 K 和畸变参数 D（5 个）
K = np.array([[571.0, 0.0, 338.2255249], [0.0, 571.0, 234.12065125], [0.0, 0.0, 1.0]])
D = np.array([0.0, 0.0, 0.0, 0.0, 0.0])
image_size = (480, 640)  # H, W
roll, pitch, yaw = 30.26064768374291, 166.3358837252657, 110.9936407676445
r = R.from_euler("ZYX", [yaw, pitch, roll], degrees=True)

# 获取对应的旋转矩阵
R_wc = r.as_matrix()
t_wc = np.array([[0.4364802 + 0.06], [0.1249509], [1.014549]])
xyz_map = generate_pixel_to_world_map(K, D, R_wc, t_wc, image_size, plane_z=0.0)

save_xyz_map_binary(xyz_map, "left_map_808.bin")
# 世界坐标系中的棋盘格角点（Z=0）

In [ ]:
# right
# 假设你已有内参 K 和畸变参数 D（5 个）
K = np.array(
    [
        [582.09985352, 0.0, 334.96206665],
        [0.0, 581.71112061, 238.98794556],
        [0.0, 0.0, 1.0],
    ]
)
D = np.array([0.0, 0.0, 0.0, 0.0, 0.0])
image_size = (480, 640)  # H, W
roll, pitch, yaw = 29.32168108797375, -166.1284918613074, 69.14326855303553
r = R.from_euler("ZYX", [yaw, pitch, roll], degrees=True)

# 获取对应的旋转矩阵
R_wc = r.as_matrix()
t_wc = np.array([[0.4476331 + 0.06], [-0.08687994], [0.9935386]])
xyz_map = generate_pixel_to_world_map(K, D, R_wc, t_wc, image_size, plane_z=0.0)

save_xyz_map_binary(xyz_map, "right_map_808.bin")
# 世界坐标系中的棋盘格角点（Z=0）

In [72]:
# front
# 假设你已有内参 K 和畸变参数 D（5 个）
K = np.array([[571.0, 0.0, 329.86688232], [0.0, 571.0, 239.08282471], [0.0, 0.0, 1.0]])
D = np.array([0.0, 0.0, 0.0, 0.0, 0.0])

# 世界坐标系中的棋盘格角点（Z=0）
obj_points = np.array(
    [
        [2.50, 0.31, -0.07],
        [2.50, -0.59, -0.07],
        [1.60, -0.59, -0.07],
        [1.60, 0.31, -0.07],
    ],
    dtype=np.float32,
)

# 图像中的像素坐标（c）
img_points = np.array(
    [
        [262, 291],
        [499, 296.0],
        [595, 464.0],
        [212, 457.0],
    ],
    dtype=np.float32,
)

R_wc = np.array(
    [
        [-0.013148, -0.349717, 0.936763],
        [-0.999875, 0.012850, -0.009237],
        [-0.008807, -0.936767, -0.349842],
    ]
)
t_wc = np.array([[0.440591], [0.029158], [0.978436]])
print(R_wc)
print(t_wc)

image_size = (480, 640)  # H, W

xyz_map = generate_pixel_to_world_map(K, D, R_wc, t_wc, image_size, plane_z=-0.07)
save_xyz_map_binary(xyz_map, "front_map_test.bin")
dist_errors, mean_error = compute_world_projection_errors(
    img_points, obj_points, xyz_map
)
for px in img_points:
    u, v = int(round(px[0])), int(round(px[1]))
    print("xyz_map:", xyz_map[v, u])

# 打印结果
print("\n📏 3D 世界坐标误差 (单位：米):")
for i, err in enumerate(dist_errors):
    print(f"  点 {i+1}: 距离误差 = {err:.4f} m")
print(f"  平均距离误差 = {mean_error:.4f} m")

[[-0.013148 -0.349717  0.936763]
 [-0.999875  0.01285  -0.009237]
 [-0.008807 -0.936767 -0.349842]]
[[0.440591]
 [0.029158]
 [0.978436]]
xyz_map: [ 2.6306937  0.2967763 -0.07     ]
xyz_map: [ 2.552405  -0.6860397 -0.07     ]
xyz_map: [ 1.5905168 -0.6502192 -0.07     ]
xyz_map: [ 1.6383387   0.32942784 -0.07      ]

📏 3D 世界坐标误差 (单位：米):
  点 1: 距离误差 = 0.1314 m
  点 2: 距离误差 = 0.1094 m
  点 3: 距离误差 = 0.0610 m
  点 4: 距离误差 = 0.0430 m
  平均距离误差 = 0.0862 m


In [ ]:
def compare_xyz_maps(
    file_path1: str,
    file_path2: str,
    height: int,
    width: int,
    rtol: float = 1e-5,
    atol: float = 1e-8,
) -> bool:
    """
    比较两个XYZ地图文件是否一致

    参数:
        file_path1, file_path2: 两个二进制文件路径
        height, width: 点云高度和宽度（需已知）
        rtol, atol: 相对和绝对容差（用于浮点数比较）

    返回:
        is_equal: 是否一致
        diff_stats: 差异统计信息（如果不一致）
    """
    # 加载两个文件
    map1 = load_xyz_map_binary(file_path1, height, width)
    map2 = load_xyz_map_binary(file_path2, height, width)

    # 检查形状和数据类型
    if map1.shape != map2.shape or map1.dtype != map2.dtype:
        return False, {
            "reason": "Shape or dtype mismatch",
            "shape1": map1.shape,
            "shape2": map2.shape,
            "dtype1": map1.dtype,
            "dtype2": map2.dtype,
        }

    # 逐元素比较（使用numpy.allclose处理浮点数精度问题）
    is_equal = np.allclose(map1, map2, rtol=rtol, atol=atol)

    if not is_equal:
        # 计算差异统计信息
        diff = np.abs(map1 - map2)
        max_diff = np.max(diff)
        mean_diff = np.mean(diff)
        diff_percentage = np.count_nonzero(diff > atol) / diff.size * 100

        return False, {
            "reason": "Element-wise mismatch",
            "max_difference": max_diff,
            "mean_difference": mean_diff,
            "difference_percentage": diff_percentage,  # 差异元素的百分比
            "example_diff": diff[np.unravel_index(np.argmax(diff), diff.shape)],
        }

    return True, {}

In [67]:
# 假设两个文件的形状都是 (480, 640, 3)
height, width = 480, 640
file1 = "/home/ubuntu/Desktop/project/sensor-calibration/notebooks/front_map_test.bin"
file2 = "/home/ubuntu/Desktop/project/sensor-calibration/tmp/front_map_wc.bin"

# 比较两个文件
is_equal, diff_info = compare_xyz_maps(file1, file2, height, width)

if is_equal:
    print("两个文件完全一致")
else:
    print("两个文件存在差异:")
    for key, value in diff_info.items():
        print(f"  {key}: {value}")

两个文件存在差异:
  reason: Element-wise mismatch
  max_difference: 11632650.0
  mean_difference: 17.696548461914062
  difference_percentage: 66.56022135416667
  example_diff: 11632650.0


In [75]:
file1 = "/home/ubuntu/Desktop/project/sensor-calibration/notebooks/front_map_test.bin"
file2 = "/home/ubuntu/Desktop/project/sensor-calibration/tmp/front_map_wc.bin"
map_1 = load_xyz_map_binary(file1, height=480, width=640)
map_2 = load_xyz_map_binary(file2, height=480, width=640)
print(map_1.shape)
print(map_2.shape)

img_points = np.array(
    [
        [262, 291],
        [499, 296.0],
        [595, 464.0],
        [212, 457.0],
    ],
    dtype=np.float32,
)
obj_points = np.array(
    [
        [2.50, 0.31, -0.07],
        [2.50, -0.59, -0.07],
        [1.60, -0.59, -0.07],
        [1.60, 0.31, -0.07],
    ],
    dtype=np.float32,
)

for px in img_points:
    u, v = int(round(px[0])), int(round(px[1]))
    print("map_1:", map_1[v, u])
    print("map_2:", map_2[v, u])
# 比较两个文件
is_equal, diff_info = compare_xyz_maps(file1, file2, height, width)

if is_equal:
    print("两个文件完全一致")
else:
    print("两个文件存在差异:")
    for key, value in diff_info.items():
        print(f"  {key}: {value}")

(480, 640, 3)
(480, 640, 3)
map_1: [ 2.6306937  0.2967763 -0.07     ]
map_2: [ 2.63069     0.29677552 -0.07      ]
map_1: [ 2.552405  -0.6860397 -0.07     ]
map_2: [ 2.5524015 -0.6860388 -0.07     ]
map_1: [ 1.5905168 -0.6502192 -0.07     ]
map_2: [ 1.5905149  -0.65021867 -0.07      ]
map_1: [ 1.6383387   0.32942784 -0.07      ]
map_2: [ 1.6383369   0.32942706 -0.07      ]
两个文件存在差异:
  reason: Element-wise mismatch
  max_difference: 11632650.0
  mean_difference: 17.696548461914062
  difference_percentage: 66.56022135416667
  example_diff: 11632650.0
